In [18]:
!pip install gensim -q
!pip install transformers -q

#install AWS dependencies
!pip install boto3
!pip install --upgrade botocore -q

     |████████████████████████████████| 132 kB 5.1 MB/s 
     |████████████████████████████████| 79 kB 9.2 MB/s 


In [2]:
import pandas as pd

BENCHMARK_FILE_PATH = '/path/to/benchmark.csv'
df = pd.read_csv(BENCHMARK_FILE_PATH)

Install 

In [3]:
!wget https://zenodo.org/record/5524358/files/edgar-w2v-200d.zip
!unzip edgar-w2v-200d.zip -d edgar-w2v

--2022-03-19 05:21:35--  https://zenodo.org/record/5524358/files/edgar-w2v-200d.zip
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75422551 (72M) [application/octet-stream]
Saving to: ‘edgar-w2v-200d.zip.1’

edgar-w2v-200d.zip. 100%[===================>]  71.93M  20.8MB/s    in 3.5s    

2022-03-19 05:21:47 (20.8 MB/s) - ‘edgar-w2v-200d.zip.1’ saved [75422551/75422551]

Archive:  edgar-w2v-200d.zip
  inflating: edgar-w2v/edgar.word.w2v.200.bin  
  inflating: edgar-w2v/edgar.word.w2v.200.vocab  


In [6]:
from gensim.models import KeyedVectors
w2v = KeyedVectors.load_word2vec_format('./edgar-w2v/edgar.word.w2v.200.bin', binary=True)

In [7]:
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D, Dense, LSTM, Conv1D, Embedding, Flatten, MaxPool1D

vocab_length = 100002
tokenizer = Tokenizer(filters="", lower=False, oov_token="<oov>")
tokenizer.fit_on_texts(w2v.index2word)
tokenizer.num_words = vocab_length

In [8]:
import tensorflow as tf
bilstm_model = tf.keras.models.load_model("sentiment-model")

In [12]:
# input length using which the model was trained
input_length = 20

# Sentenced on which benchmarking is to be run
X = df.Text

# Tokenize and pad the sentences upto input_length
X = pad_sequences(tokenizer.texts_to_sequences(X), maxlen=input_length)

# Get the predictions on the sentences
preds = bilstm_model.predict(X)

In [13]:
# Get the index of the prediction with maximum probability
preds = np.argmax(preds, axis=1).tolist()

# Labels for sentiment analysis
labels = ['negative', 'neutral', 'positive']

# Convert the outputs from predictions to proper output labels
df['BiLSTM'] = list(map(lambda x: labels[x], preds))

## FinBERT, FinancialBERT, DistilRoBERTa

Loading the models

In [15]:
# importing the required libraries
# importing all the required libraries
import pandas as pd
import numpy as np
import os, json
from tqdm import tqdm
import nltk
from scipy import stats
from nltk.tokenize import sent_tokenize
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import regex as re
nltk.download('punkt')
nltk.download('stopwords')

# all the three models used for voting
tokenizer1 = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model1 = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
classifier1 = pipeline("sentiment-analysis", model=model1, tokenizer=tokenizer1)

tokenizer2 = AutoTokenizer.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis")
model2 = AutoModelForSequenceClassification.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis")
classifier2 = pipeline("sentiment-analysis", model=model2, tokenizer=tokenizer2)

tokenizer3 = AutoTokenizer.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
model3 = AutoModelForSequenceClassification.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
classifier3 = pipeline("sentiment-analysis", model=model3, tokenizer=tokenizer3)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/758 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/933 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/313M [00:00<?, ?B/s]

Inferencing from the models

In [16]:
scores_1 = classifier1(list(df['Text'].values))
scores_2 = classifier2(list(df['Text'].values))
scores_3 = classifier3(list(df['Text'].values))

labels_1 = []
labels_2 = []
labels_3 = []

for i,j,k in zip(scores_1,scores_2,scores_3):
  labels_1.append(i['label'])
  labels_2.append(j['label'])
  labels_3.append(k['label'])

df["Prosus_FinBERT_preds"] = labels_1
df["FinancialBERT_preds"] = labels_2
df["DistilRoBERTa_finetuned_preds"] = labels_3
df.head()

,Text,Sentiment,Prosus_FinBERT_preds,FinancialBERT_preds,DistilRoBERTa_finetuned_preds,BiLSTM
0,We are a leading cloud-based platform for work...,positive,positive,positive,positive,positive
1,We were founded in 2005 with a vision to build...,positive,neutral,neutral,neutral,positive
2,We generate revenue primarily from the sale of...,neutral,neutral,neutral,neutral,neutral
3,We maintain a valuation allowance on our U.S. ...,neutral,neutral,neutral,neutral,negative
4,We recorded share-based compensation expense f...,positive,negative,neutral,neutral,neutral


AWS Comprehend

In [ ]:
# Set the following AWS credentials using AWS CLI
# ACCESS_KEY
# SECRET_KEY
# DEFAULT-REGION

In [1]:
import boto3
client = boto3.client('comprehend')

In [ ]:
res = []
for text in df.Text:
  response = client.detect_sentiment(
      Text=text,
      LanguageCode='en'
  )

  gcategory, gscore = '', -1
  for category, score in response['SentimentScore'].items():
    if score > gscore:
      gscore = score
      gcategory = category
  res.append(gcategory)

df['AWS'] = list(map(lambda x: x.lower(), res))

Evaluation Metrics

In [ ]:
import pandas as pd
df = pd.read_csv('sentiments.csv')

In [ ]:
df.head()

,Text,Sentiment,Prosus_FinBERT_preds,FinancialBERT_preds,DistilRoBERTa_finetuned_preds,AWS,BiLSTM
0,We are a leading cloud-based platform for work...,positive,positive,positive,positive,neutral,positive
1,We were founded in 2005 with a vision to build...,positive,neutral,neutral,neutral,neutral,positive
2,We generate revenue primarily from the sale of...,neutral,neutral,neutral,neutral,neutral,neutral
3,We maintain a valuation allowance on our U.S. ...,neutral,neutral,neutral,neutral,neutral,negative
4,We recorded share-based compensation expense f...,positive,negative,neutral,neutral,neutral,neutral


In [ ]:
df.columns

Index(['Text', 'Sentiment', 'Prosus_FinBERT_preds', 'FinancialBERT_preds',
       'DistilRoBERTa_finetuned_preds', 'AWS', 'BiLSTM'],
      dtype='object')

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

data = {
    'model'    : [],
    'accuracy' : [],
    'precision': [],
    'recall'   : [],
    'F1-score' : []
}

for col in df.columns[2:]:
  acc = accuracy_score(df['Sentiment'], df[col])
  prc = precision_score(df['Sentiment'], df[col], average='macro') 
  rec = recall_score(df['Sentiment'], df[col], average='macro') 

  f1_score = 2 * prc * rec / (prc + rec)

  print(f'Accuracy Score of {col} : {acc}')
  print(f'Precision Score of {col}: {prc}')
  print(f'Recall Score of {col}   : {rec}')
  print(f'F1 Score of {col}       : {f1_score}')

  data['model'].append(col)
  data['accuracy'].append(acc)
  data['precision'].append(prc)
  data['recall'].append(rec)
  data['F1-score'].append(f1_score)

  print()

Accuracy Score of Prosus_FinBERT_preds : 0.6080402010050251
Precision Score of Prosus_FinBERT_preds: 0.6200673400673401
Recall Score of Prosus_FinBERT_preds   : 0.5949186009026435
F1 Score of Prosus_FinBERT_preds       : 0.6072326962463166

Accuracy Score of FinancialBERT_preds : 0.628140703517588
Precision Score of FinancialBERT_preds: 0.671302606408417
Recall Score of FinancialBERT_preds   : 0.6195196647324307
F1 Score of FinancialBERT_preds       : 0.6443724669990144

Accuracy Score of DistilRoBERTa_finetuned_preds : 0.5979899497487438
Precision Score of DistilRoBERTa_finetuned_preds: 0.622860791826309
Recall Score of DistilRoBERTa_finetuned_preds   : 0.5769664732430689
F1 Score of DistilRoBERTa_finetuned_preds       : 0.5990358859875232

Accuracy Score of AWS : 0.32663316582914576
Precision Score of AWS: 0.44047619047619047
Recall Score of AWS   : 0.33570075757575757
F1 Score of AWS       : 0.3810167029774873

Accuracy Score of BiLSTM : 0.5025125628140703
Precision Score of BiLSTM:

In [ ]:
pd.DataFrame(data).to_csv('eval.csv', index=False)